In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
import numpy as np
import gym
import gym.spaces
from pylab import *
import scipy.misc
import time
from collections import namedtuple

from train_ops import create_train_ops
from utils import *

In [3]:
%matplotlib notebook

In [4]:
G = 0.99
N_ACTIONS = 3
ACTIONS = np.arange(N_ACTIONS) + 1

## Network setup

In [5]:
Network = namedtuple('Network', 's a r a_softmax graph_v policy_loss value_loss')

In [6]:
def create_network(scope):
    with tf.variable_scope(scope):
        graph_s = tf.placeholder(tf.float32, [None, 84, 84, 4])
        graph_action = tf.placeholder(tf.int64, [None])
        graph_r = tf.placeholder(tf.float32, [None])

        x = tf.layers.conv2d(
                inputs=graph_s,
                filters=32,
                kernel_size=8,
                strides=4,
                activation=tf.nn.relu)

        x = tf.layers.conv2d(
                inputs=x,
                filters=64,
                kernel_size=4,
                strides=2,
                activation=tf.nn.relu)

        x = tf.layers.conv2d(
                inputs=x,
                filters=64,
                kernel_size=3,
                strides=1,
                activation=tf.nn.relu)

        w, h, f = x.shape[1:]
        x = tf.reshape(x, [-1, int(w * h * f)])

        x = tf.layers.dense(
                inputs=x,
                units=512,
                activation=tf.nn.relu)

        a_logits = tf.layers.dense(
                inputs=x,
                units=N_ACTIONS,
                activation=None)

        a_softmax = tf.nn.softmax(a_logits)

        graph_v = tf.layers.dense(
            inputs=x,
            units=1,
            activation=None)
        #graph_v = graph_v[:, 0]

        p = 0
        for i in range(N_ACTIONS):
            p += tf.cast(tf.equal(graph_action, i), tf.float32) * a_softmax[:, i]
        nlp = -1 * tf.log(p)
        policy_loss = tf.reduce_mean(nlp * graph_r)

        value_loss = tf.reduce_mean((graph_r - graph_v) ** 2)
        
        network = Network(
            s=graph_s,
            a=graph_action,
            r=graph_r,
            a_softmax=a_softmax,
            graph_v=graph_v,
            policy_loss=policy_loss,
            value_loss=value_loss)
        
        return network

In [ ]:
tf.summary.scalar('loss', loss)
tf.summary.scalar('loss2', loss2)
summaries = tf.summary.merge_all()
summary_writer = tf.summary.FileWriter('summaries/')

In [7]:
def list_set(l, i, val):
    assert(len(l) == i)
    l[i] = val

In [18]:
class Worker:
    
    def __init__(self, worker_n, env_name):
        self.env = gym.make(env_name)
        
        worker_scope = "worker_%d" % worker_n
        self.network = create_network(worker_scope)
        self.scope = worker_scope
        
        # TODO: do these need to be separate
        policy_optimizer = tf.train.AdamOptimizer()
        value_optimizer = tf.train.AdamOptimizer()
        
        self.update_policy_gradients, self.apply_policy_gradients, self.zero_policy_gradients = \
            create_train_ops(self.network.policy_loss,
                             policy_optimizer,
                             update_scope=worker_scope,
                             apply_scope='global')
        
        self.update_value_gradients, self.apply_value_gradients, self.zero_value_gradients = \
            create_train_ops(self.network.value_loss,
                             value_optimizer,
                             update_scope=worker_scope,
                             apply_scope='global')
        
        self.env.reset()

        self.t_max = 100

        
    def run_episode(self):
        states = []
        actions = []
        rewards = []
        i = 0
        
        sess.run([self.zero_policy_gradients,
                  self.zero_value_gradients])
        copy_network(sess,
                     from_scope='global',
                     to_scope=self.scope)

        o, _, _ = utils.get_o(self.env, 0) # 0 = do nothing (in Pong, at least)
        list_set(states, i, o)

        done = False
        while not done and i < self.t_max:
            feed_dict = {self.network.graph_s: [o]}
            a_p = sess.run(a_softmax, feed_dict=feed_dict)[0]
            a = np.random.choice(ACTIONS, p=a_p)
            list_set(actions, i, a)

            o, r, done = utils.get_o(self.env, a)
            list_set(rewards, i, r)
            list_set(states, i + 1, o)

            i += 1

        if done:
            r = 0
        else:
            feed_dict = {self.network.s: [list_get(states, self.t)]}
            r = sess.run(self.network.graph_v, feed_dict=feed_dict)

        for j in range(i, -1, -1):
            r = rewards[j] + G * r
            feed_dict = {self.network.s: states[j],
                         self.network.a: actions[j] - 1, # map from possible actions (1, 2, 3) -> (0, 1, 2)
                         self.network.r: r}
            sess.run([self.update_policy_gradients,
                      self.update_value_gradients],
                      feed_dict=feed_dict)
        sess.run([self.apply_policy_gradients,
                  self.apply_value_gradients])
        sess.run([self.zero_policy_gradients,
                  self.zero_value_gradients])

In [19]:
tf.reset_default_graph()
sess = tf.Session()

In [20]:
global_network = create_network('global')

In [ ]:
workers = []
for i in range(4):
    worker = Worker(i, 'Pong-v0')

[2017-07-23 19:11:55,023] Making new env: Pong-v0
[2017-07-23 19:11:58,313] Making new env: Pong-v0


## Action probability selection test

In [ ]:
sess.run(tf.global_variables_initializer())
s = np.random.random((1, 84, 84, 4))
s = np.vstack([s] * N_ACTIONS)
expected = -1 * np.log(sess.run(a_softmax, feed_dict={graph_s: s})[0])
actual = sess.run(nlp, feed_dict={graph_s: s, graph_action: np.arange(N_ACTIONS)})
np.testing.assert_allclose(expected, actual)

## Basic policy network training test

In [ ]:
sess.run(tf.global_variables_initializer())
n = 5
s = np.random.random((1, 84, 84, 4))
s = np.vstack([s] * n)

# encourage action 0
print(sess.run(a_softmax, feed_dict={graph_s: s})[0])
a = [0] * n
ad = [1] * n
for i in range(3):
    sess.run(t, feed_dict={graph_s: s, graph_action: a, graph_advantage: ad})
    print(sess.run(a_softmax, feed_dict={graph_s: s})[0])
    
# now the probability of action[0] will be very close to 1,
# so the gradients will be too small to work
# so reset
sess.run(tf.global_variables_initializer())
print()

# discourage action 0
print(sess.run(a_softmax, feed_dict={graph_s: s})[0])
a = [0] * n
ad = [-1] * n
for i in range(3):
    sess.run(t, feed_dict={graph_s: s, graph_action: a, graph_advantage: ad})
    print(sess.run(a_softmax, feed_dict={graph_s: s})[0])

In [ ]:
sess.run(tf.global_variables_initializer())
sess.run(tf.assign(graph_lr, 1e-3))
print(sess.run(a_softmax, feed_dict={graph_s: s})[0])

In [ ]:
sess.run(t1, feed_dict={graph_s: s, graph_action: [0], graph_advantage: [-1]})
print(sess.run(a_softmax, feed_dict={graph_s: s})[0])

## Basic value network training test

In [ ]:
s = np.random.normal(size=(5, 84, 84, 4))
r = np.array([1, 2, 3, 4, 5]) / 5.0
print(r)
sess.run(tf.global_variables_initializer())
sess.run(graph_v, feed_dict={graph_s: s})

In [ ]:
sess.run(t2, feed_dict={graph_s: s, graph_r: r})
sess.run([loss2, graph_v], feed_dict={graph_s: s, graph_r: r})

## Dummy environment

In [ ]:
class DummyEnv:
    action_space = gym.spaces.discrete.Discrete(2)
    N_STATES = 3
    N_REWARDED_SEQUENCES = int(N_ACTIONS ** N_STATES / 10)
    
    def __init__(self):
        self.rewarded_sequences = np.clip(np.random.normal(loc=int(N_ACTIONS / 2), size=(self.N_REWARDED_SEQUENCES, self.N_STATES)).astype(np.int), 0, N_ACTIONS)
        self.rewarded_sequences = np.unique(self.rewarded_sequences, axis=0)
        self.o = {}
        # look at all unique sequence prefixes
        for i in range(1, self.N_STATES + 1):
            prefixes = np.unique(env.rewarded_sequences[:, :i], axis=0)
            for p in prefixes:
                self.o[tuple(p)] = np.random.normal(size=(84, 84, 4))
        self.default_o = np.random.normal(size=(84, 84, 4))
        self.init_o = np.random.normal(size=(84, 84, 4))
    
    def reset(self):
        self.i = 0
        self.history = []
        self.possible_sequences = np.copy(self.rewarded_sequences)
        return self.init_o
    
    def step(self, a):
        self.history.append(a)
        
        ps_new = []
        l = len(self.history)
        for seq in self.possible_sequences:
            if np.array_equal(seq[:l], self.history):
                ps_new.append(seq)
        self.possible_sequences = ps_new
        if len(self.possible_sequences) > 0:
            reward = 1
        else:
            reward = 0
            
        if tuple(self.history) in self.o:
            o = self.o[tuple(self.history)]
        else:
            o = self.default_o

        if self.i != self.N_STATES - 1:
            done = False
            self.i += 1
        else:
            done = True

        return o, reward, done, None

In [ ]:
env = DummyEnv()

In [ ]:
for i in range(N_ACTIONS):
    env.reset()
    env.step(i)
    print(len(env.possible_sequences))

In [ ]:
# Initial state same?
o = env.reset()
print(np.sum(o))
o = env.reset()
print(np.sum(o))

In [ ]:
# Default state the same?
env.reset()
o, _, _, _ = env.step(0)
print(np.sum(o))
env.reset()
o, _, _, _ = env.step(8)
print(np.sum(o))

In [ ]:
# Always the same state returned for a given sequence?
env.reset()
o, _, _, _ = env.step(5)
print(np.sum(o))
env.reset()
o, _, _, _ = env.step(5)
print(np.sum(o))

env.reset()
env.step(5)
o, _, _, _ = env.step(5)
print(np.sum(o))
env.reset()
env.step(5)
o, _, _, _ = env.step(5)
print(np.sum(o))

In [ ]:
def test_seq(seq):
    for a in seq:
        print("Took action", a)
        _, reward, done, _ = env.step(a)
        print("Got reward ", reward)
        print("Done:", done)

env.reset()
test_seq([0, 0, 0])
print()
env.reset()
test_seq(env.rewarded_sequences[0])

## Testing proper

### Manual testing

In [ ]:
sess.run(tf.global_variables_initializer())
sess.run(t, feed_dict={graph_s: [env.o[2]], graph_action: [1], graph_advantage: [1]})
sess.run(a_softmax, feed_dict={graph_s: env.o})

### Testing with environment

In [ ]:
rpe = []
rpe2 = []
sess.run(tf.global_variables_initializer())
steps = 0

In [ ]:
def run(n_episodes=1000):
    global steps
    for i in range(n_episodes):
        if i % 100 == 0:
            print(i)
        env.reset()
        o, _, _ = get_o(0) # do nothing
        done = False
        episode_rewards = []
        rewards = []
        actions = []
        states = []
        while not done:
            states.append(o)
            a_p = sess.run(a_softmax, feed_dict={graph_s: [o]})[0]
            a = np.random.choice(ACTIONS, p=a_p)
            actions.append(a)

            o, r, done = get_o(a)
            rewards.append(r)

            if r != 0:
                episode_rewards.append(r)
                returns = rewards_to_returns(rewards, G=0.9)
                actions = np.array(actions) - 1 # map from possible actions (1, 2, 3) -> (0, 1, 2)
                summ, _, _ = sess.run([summaries, t, t2], feed_dict={graph_s: states, graph_action: actions, graph_r: returns})
                summary_writer.add_summary(summ, steps)
                rewards = []
                actions = []
                states = []
                steps += 1
        
        rpe2.append(episode_rewards)
        if len(rpe) is 0:
            rpe.append(np.sum(episode_rewards))
        else:
            rpe.append(rpe[-1] + 0.01 * (np.sum(episode_rewards) - rpe[-1]))

In [ ]:
RENDER = True
run(n_episodes=1)

In [ ]:
rpe = [np.sum(rpe2[0])]
for s in rpe2[1:]:
    rpe.append(rpe[-1] + 0.01 * (np.sum(s) - rpe[-1]))

In [ ]:
plt.plot(rpe)

## Old code

In [ ]:
t = 0
t_max = 100

s = np.zeros(10)
r1 = np.zeros(10)
a = np.zeros(10)
r2_list = []

for i in range(10):
    t_start = t
    s[t] = env.reset()
    done = False
    
    while True:
        a_p = sess.run(ap_op, feed_dict={s_op: s})
        a[t] = np.random.choice(ACTIONS, p=a_p)
        s[t+1], r[t], done, _ = env.step(a[t])
        t += 1        
    if done or t - t_start == t_max:
        break
    
    if done:
        r2 = 0
    else:
        r2 = sess.run(v_op, feed_dict={s_op: s[t]})
    
    for i in range(t-1, t_start-1, -1):
        r2_list.append(r[i] + g * r2)
    
    
    sess.run(t1, feed_dict={s_op: s, big_r_op: r})
    sess.run(t2, feed_dict={s_op: s, big_r_op: r})

In [ ]:
observation.shape
action = env.action_space.sample()
os = []
for i in range(4):
    o1, _, _, _ = env.step(action)
    o2, _, _, _ = env.step(action)
    o = np.maximum(o1, o2)
    o = np.mean(o, axis=2)
    o = scipy.misc.imresize(o, (84, 84))
    os.append(o)
os = np.stack(os, axis=-1)

In [ ]:
os = np.resize(os, (1, 84, 84, 4))
print(os.shape)
print(sess.run(q['network1'], feed_dict={s: os}))
print(sess.run(qi['network1'], feed_dict={s: os}))